# Пример связки GRASS GIS и sklearn для задачи классификации

## Обучающая выборка

Сначала посмотрим на то, насколько вообще можно обучить модель, попытавшись переобучить ее. Т.е. для начала обучим систему без разбиения выборки на обучающую и тестовую, получившийся результат даст представление о верхней границе качества.

In [1]:
import os
import pickle

import pandas as pd
import numpy as np
import sklearn as sk

from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix

from sklearn.model_selection import GridSearchCV

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [2]:
import grasslib

from grasslib import GRASS

In [3]:
# os.environ['LD_LIBRARY_PATH'] = '/usr/lib/grass74/lib'
grs = GRASS(gisbase='/usr/lib/grass74', 
            dbase='/home/klsvd/GRASSDATA',
            location='Landsat',
            mapset='PERMANENT'
)

## Обучающая выборка

Создадим обучающую выборку, которая будет классифицировать данные по рубкам за зиму 15-16 годов.

In [4]:
print grs.grass.read_command('g.region', rast='B1', flags='p')

projection: 1 (UTM)
zone:       19
datum:      wgs84
ellipsoid:  wgs84
north:      -1162785
south:      -1395315
west:       429885
east:       658515
nsres:      30
ewres:      30
rows:       7751
cols:       7621
cells:      59070371



Склеим слои с обучающими полигонами:

In [5]:
grs.grass.run_command('v.to.rast', 
                      input='test1', output='test1', 
                      use='val', value=1, 
                      overwrite=True)
grs.grass.run_command('v.to.rast', 
                      input='test2', output='test2', 
                      use='val', value=2, 
                      overwrite=True)
grs.grass.run_command('r.patch', input='test1,test2', output='train', overwrite=True)

print grs.grass.read_command('r.report', map='train', units='c')

+-----------------------------------------------------------------------------+
|                         RASTER MAP CATEGORY REPORT                          |
|LOCATION: Landsat                                    Wed Aug 22 21:10:27 2018|
|-----------------------------------------------------------------------------|
|          north: -1162785    east: 658515                                    |
|REGION    south: -1395315    west: 429885                                    |
|          res:         30    res:      30                                    |
|-----------------------------------------------------------------------------|
|MASK: none                                                                   |
|-----------------------------------------------------------------------------|
|MAP: Rasterized vector map from values (train in PERMANENT)                  |
|-----------------------------------------------------------------------------|
|                       Category Informa

Соберем входные данные (каналы растров), которые будут использоваться для обучения:

In [6]:
inputs = grs.grass.list_strings('rast', pattern="toar*")
inputs

['toar1@PERMANENT',
 'toar10@PERMANENT',
 'toar11@PERMANENT',
 'toar2@PERMANENT',
 'toar3@PERMANENT',
 'toar4@PERMANENT',
 'toar5@PERMANENT',
 'toar6@PERMANENT',
 'toar7@PERMANENT',
 'toar8@PERMANENT',
 'toar9@PERMANENT']

In [7]:
output=['train']

Изменим разрешение растров для скорости работы:

In [8]:
print grs.grass.read_command('g.region', rast='B1', res=90, flags='p')

projection: 1 (UTM)
zone:       19
datum:      wgs84
ellipsoid:  wgs84
north:      -1162785
south:      -1395315
west:       429885
east:       658515
nsres:      89.98839009
ewres:      90.01181102
rows:       2584
cols:       2540
cells:      6563360



In [9]:
data = grs.rasters_to_array(inputs+output)
data.shape

(6563360, 12)

Были считаны пиксели, содержащие no-data, удалим их из обучающей выборки.

In [10]:
good_rows = np.all(~np.isnan(data), axis=1)
print(good_rows)

[False False False ..., False False False]


In [11]:
data = data[good_rows, ]
data.shape

(588, 12)

In [12]:
X_train = data[:, :-1]
print(X_train.shape)
y_train = data[:, -1]
print(y_train.shape)

(588, 11)
(588,)


## Случайный лес

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
forest = RandomForestClassifier(n_estimators=10, max_depth=3, random_state=1, n_jobs=3)

In [15]:
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=3, oob_score=False, random_state=1,
            verbose=0, warm_start=False)

In [16]:
y_pred = forest.predict(X_train)
print confusion_matrix(y_train, y_pred)

[[ 48   2]
 [  0 538]]


In [17]:
X = grs.rasters_to_array(inputs)
y = forest.predict(X)

grs.grass.run_command('g.remove', type='rast', name='forest.result90m', flags='f')
grs.array_to_rast(arr=y, map_name='forest.result90m')

In [18]:
grs.grass.run_command('r.out.gdal', input='forest.result90m', output='forest.result90m.tif', overwrite=True)

0

## Обучение с перекрестной проверкой

Обучим модель "по-правильному" с использованием перекрестной проверки и тестового множества.

In [19]:
data = grs.rasters_to_array(inputs+output)
good_rows = np.all(~np.isnan(data), axis=1)
data = data[good_rows, ]

X = data[:, :-1]
y = data[:, -1]

X.shape

(588, 11)

Собственно разбиение на множества:

In [20]:
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.33, random_state=1)

In [21]:
X_train.shape

(393, 11)

Обучим одну модель (случайный лес, параметры по умолчанию), оценим ошибку и ее разброс при помощи перекрестной проверки:

In [22]:
pipe_forest = Pipeline([
        # ('pca', PCA(n_components=10)), 
        ('clf', RandomForestClassifier(random_state=1, max_depth=3, n_jobs=3))
])


scores = sk.model_selection.cross_val_score(
    estimator=pipe_forest,
    X=X_train,
    y=y_train,
    cv=10
)

print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores) * 2))

CV accuracy scores: [ 1.          0.975       0.975       1.          1.          0.97435897
  0.97435897  1.          1.          1.        ]
CV accuracy: 0.990 +/- 0.025


Проверим качество на теством множестве:

In [23]:
pipe_forest.fit(X_train, y_train)
y_pred = pipe_forest.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 15,   0],
       [  0, 180]])

При помощи перекрестной проверки найдем модель с оптимальным числом деревьев в лесу:

In [24]:
param_range = [3, 5, 9, 15]
param_grid = [{'clf__n_estimators': param_range}]
gs = GridSearchCV(
    estimator=pipe_forest,
    param_grid=param_grid,
    scoring='accuracy',
    cv=10
)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.989821882952
{'clf__n_estimators': 3}


### Прогноз

Проверим результат на тестовом множестве:

In [25]:
best_model = gs.best_estimator_
y_pred = best_model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 15,   0],
       [  0, 180]])

Запустим работу модели на всех данных, сохраним результаты в растры:

In [26]:
X = grs.rasters_to_array(inputs)
y = best_model.predict(X)

grs.grass.run_command('g.remove', type='rast', name='forest1.result90m', flags='f')
grs.array_to_rast(arr=y, map_name='forest1.result90m')

grs.grass.run_command('r.out.gdal', input='forest1.result90m', output='forest1.result90m.tif', overwrite=True)

0

## Всеми любимые нейросети

In [27]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [28]:
data = grs.rasters_to_array(inputs+output)
good_rows = np.all(~np.isnan(data), axis=1)
data = data[good_rows, ]

X = data[:, :-1]
y = data[:, -1]

X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.33, random_state=1)

In [29]:
pipe_net = Pipeline([
        ('norm', StandardScaler()),
        ('clf', MLPClassifier(max_iter=200, hidden_layer_sizes=(5, 2), alpha=0.01, random_state=1))
])

scores = sk.model_selection.cross_val_score(
    estimator=pipe_net,
    X=X_train,
    y=y_train,
    cv=10
)

print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores) * 2))

/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


CV accuracy scores: [ 0.9         0.825       0.8         0.825       0.825       0.8974359
  0.87179487  0.84615385  0.84210526  0.89473684]
CV accuracy: 0.853 +/- 0.068


In [30]:
pipe_net.fit(X_train, y_train)
y_pred = pipe_net.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 15,   0],
       [ 44, 136]])

Оптимальное число слоев:

In [31]:
param_range = [(3, ), (10,), (20,), (3, 2), (5, 3), (9, 5), (15, 7)]
param_grid = [{'clf__hidden_layer_sizes': param_range}]
gs = GridSearchCV(
    estimator=pipe_net,
    param_grid=param_grid,
    scoring='accuracy',
    cv=10
)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.989821882952
{'clf__hidden_layer_sizes': (10,)}


In [32]:
best_model = gs.best_estimator_
y_pred = best_model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 15,   0],
       [  0, 180]])

In [33]:
X = grs.rasters_to_array(inputs)
y = best_model.predict(X)

grs.grass.run_command('g.remove', type='rast', name='net.result90m', flags='f')
grs.array_to_rast(arr=y, map_name='net.result90m')

grs.grass.run_command('r.out.gdal', input='net.result90m', output='net.result90m.tif', overwrite=True)

0